In [53]:
from src.features.objects import object as object_helper
from src.models.lda import lda as lda_helper
from src import constants

import pandas as pd
import numpy as np

import psycopg2

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
object_occurence_rate = object_helper.get_occurence_rate()
object_occurence_rate.head()

,platform,id,person,bicycle,car,motorbike,aeroplane,bus,train,truck,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,facebook,100001732932719/1647247215343027,0.000036,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29510
1,facebook,100002259829821/1786627524755891,0.000155,0.0,0.000997,0.000000,0.0,0.000000,0.0,0.000067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,816264
2,facebook,100002259829821/1786745844744059,0.003062,0.0,0.002931,0.000000,0.0,0.000000,0.0,0.000022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55911
3,facebook,100004527049328/972499912910901,0.000395,0.0,0.000122,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177064
4,facebook,100004527049328/973448889482670,0.000672,0.0,0.001532,0.000004,0.0,0.000008,0.0,0.000545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138453


In [55]:
features_connection = psycopg2.connect(database="video_article_retrieval", user="postgres")
average_topics = pd.read_sql("SELECT * FROM average_topics", con=features_connection)
average_topics.head()

,id,platform,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,fox26houston/10157181811750348,facebook,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,0.000000,0.00393
1,jessica.salazar.338/2111423425558048,facebook,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,0.543781,0.00000
2,JoyNewsOnTV/2167529583318936,facebook,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,0.017435,0.00000
3,100009629315589/737936519870652,facebook,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,0.106360,0.00000
4,185998671553/10157087493556554,facebook,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,0.000000,0.00000


In [56]:
# Right join on the y dataframe to only keep the videos where we have detected the objects
data = pd.merge(average_topics, object_occurence_rate,  how='right', left_on=['id','platform'], right_on = ['id','platform'])
data.head()

,id,platform,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,fox26houston/10157181811750348,facebook,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,449536
1,jessica.salazar.338/2111423425558048,facebook,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82047
2,JoyNewsOnTV/2167529583318936,facebook,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196371
3,100009629315589/737936519870652,facebook,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,218823
4,185998671553/10157087493556554,facebook,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38380


In [57]:
topics = lda_helper.get_topic_columns(10)
objects = [label.replace(" ", "_") for label in constants.COCO_CLASS_NAMES]
data[topics].head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,0.0,0.767010,0.0,0.153838,0.046777,0.000000,0.005562,0.020166,0.000000,0.00393
1,0.0,0.099226,0.0,0.158874,0.159987,0.016518,0.000000,0.020300,0.543781,0.00000
2,0.0,0.000000,0.0,0.000000,0.304423,0.000000,0.675539,0.000000,0.017435,0.00000
3,0.0,0.546476,0.0,0.000000,0.138599,0.000000,0.000000,0.201420,0.106360,0.00000
4,0.0,0.000000,0.0,0.264172,0.359579,0.117422,0.000000,0.255583,0.000000,0.00000


In [58]:
data[objects].head()

,person,bicycle,car,motorbike,aeroplane,bus,train,truck,boat,traffic_light,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush
0,0.000374,0.0,0.00071,0.0,0.0,0.000005,0.0,0.000207,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.001757,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001278,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000241,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000098,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
data = data.sample(frac=1).reset_index(drop=True)

In [99]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor

X_train, X_test, y_train, y_test = train_test_split(data[objects], data[topics], test_size=0.2)

# Get the performance values for guessing the mean
regr = DummyRegressor(strategy='mean')

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.02
Variance score: -0.00


In [100]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.26
Variance score: -11.31


In [ ]:
models = [linear_model.LinearRegression()]
for model in models:
    model.fit(data[objects], data[topics])
    scores = cross_val_score(model, data[objects], data[topics], cv=5)
    # print('Coefficients: \n', regr.coef_)
    # print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))
    # print(model.score(X_test, y_test))
    print(scores)